In [1]:
from itertools import combinations
from collections import namedtuple
import numpy as np
from icecream import ic
import random

## Simple Test Problem

In [2]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('lab2/test_problem.npy')
problem = test_problem


In [9]:
Individual = namedtuple('individual', ['genotype', 'weight'])

## Common tests

In [136]:
problem = np.load('lab2/problem_g_20.npy')

In [137]:
# Negative values?
np.any(problem < 0)

np.False_

In [6]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

False

In [7]:
# Symmetric matrix?
np.allclose(problem, problem.T)

False

In [8]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

False

# Solution

In [ ]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 20
MUTATION_RATE = 0.2

In [11]:
# Create a greedy solution starting from a random node
def greedy_solution(problem: np.ndarray, start_node: int, randomization: float = 0.2) -> list:
    solution = [start_node]
    unvisited_nodes = list(range(problem.shape[0]))
    unvisited_nodes.remove(start_node)
    while len(unvisited_nodes) > 0:
        if np.random.rand() < randomization:
            next_node = int(np.random.choice(list(unvisited_nodes)))
        else:
            last_node = solution[-1]
            min_distance = float('inf')
            next_node = None
            for node in unvisited_nodes:
                distance = problem[last_node, node]
                if distance < min_distance:
                    min_distance = distance
                    next_node = node
        
        unvisited_nodes.remove(next_node)
        solution.append(next_node)
    
    return solution


In [187]:
def greedy_solution_2(problem: np.ndarray, start_node: int, randomization: float = 0.2) -> list:
    # Given a node A instead of finding the closest node we check among all the nodes and we select as the next one
    # the one who has node A as the closest one. This way we reduce the chances of leaving an isolated node at the end.
    solution = [start_node]
    unvisited_nodes = list(range(problem.shape[0]))
    unvisited_nodes.remove(start_node)
    while len(unvisited_nodes) > 0:
        if np.random.rand() < randomization:
            next_node = int(np.random.choice(list(unvisited_nodes)))
        
        else:
            possible_next_nodes = []        # define all possible nodes which can be next
            for node in unvisited_nodes:    # iterate on every unvisited node
                
                closest = None
                min_distance = float('inf')

                for other_node in unvisited_nodes + [solution[0]]:      # for each unvisited node find its closest node (we also consider the first node since we want to close the loop)
                    if node != other_node:
                        distance = problem[node, other_node]
                        if distance < min_distance:
                            min_distance = distance
                            closest = other_node
                
                if closest == solution[-1]:             # if the closest node is the last node in the solution, then this node can be the next one
                    possible_next_nodes.append(node)

            if len(possible_next_nodes) == 0:         # if no possible next nodes found, fallback to standard greedy
                last_node = solution[-1]
                min_distance = float('inf')
                next_node = None
                for node in unvisited_nodes:
                    distance = problem[last_node, node]
                    if distance < min_distance:
                        min_distance = distance
                        next_node = node
            else:
                # from the possible next nodes select the closest one from the last node in the solution
                last_node = solution[-1]
                min_distance = float('inf')
                next_node = None
                for node in possible_next_nodes:
                    distance = problem[last_node, node]
                    if distance < min_distance:
                        min_distance = distance
                        next_node = node
            
                

        unvisited_nodes.remove(next_node)
        solution.append(next_node)
    
    return solution

In [35]:
def compute_weight(problem: np.ndarray, solution: list) -> float:
    weight = 0.0
    for i in range(len(solution)-1):
        weight += problem[solution[i], solution[i+1]]
    weight += problem[solution[-1], solution[0]]  # Return to start
    return weight

def mutation(solution: Individual, problem: np.ndarray, n_mutations: int = 1) -> Individual:
    new_solution = solution.genotype.copy()
    for _ in range(n_mutations):
        # Select two different indices
        nodes = random.sample(range(len(solution)), 2)
        # Swap
        new_solution[nodes[0]], new_solution[nodes[1]] = new_solution[nodes[1]], new_solution[nodes[0]]

    # re-normalize
    idx_start = new_solution.index(0)
    new_solution = new_solution[idx_start:] + new_solution[:idx_start]

    return Individual(genotype=new_solution, weight=compute_weight(problem, new_solution))

def crossover(parent1: Individual, parent2: Individual, problem: np.ndarray) -> Individual:
    # TODO: test if this is a good crossover method
    new_solution = []
    index1 = 1
    index2 = random.randint(1, len(parent2.genotype)-1)     # start from a random position in parent2
    turn = 1

    # both parents start with the same node (0)
    new_solution.append(parent1.genotype[0])
    not_included = list(range(1, len(parent1.genotype)))

    while True:
        # pick nodes drom parent 1, then when we encounter the node which is in parent2.genotype[index2] we start picking from parent 2
        # Then again we switch when we encounter the node which is in parent1.genotype[index1]
        if turn%2 == 1:
            if parent1.genotype[index1] in not_included:
                new_solution.append(parent1.genotype[index1])
                not_included.remove(parent1.genotype[index1])
            if parent1.genotype[index1] == parent2.genotype[index2]:
                turn += 1
            index1 = (index1 + 1) % len(parent1.genotype)
        else:
            if parent2.genotype[index2] in not_included:
                new_solution.append(parent2.genotype[index2])
                not_included.remove(parent2.genotype[index2])
            if parent2.genotype[index2] == parent1.genotype[index1]:
                turn += 1
            index2 = (index2 + 1) % len(parent2.genotype)

        if len(new_solution) == len(parent1.genotype):
            break

    return Individual(genotype=new_solution, weight=compute_weight(problem, new_solution))

In [124]:
def insert_mutation(solution: Individual, problem: np.ndarray, n_mutations: int = 1) -> Individual:
    # this mutation has the goal of minimizing the chenges in order, trying to preserve the original one
    new_solution = solution.genotype.copy()
    for _ in range(n_mutations):
        nodes = random.sample(range(len(solution.genotype)), 2)
        nodes.sort()
        new_solution = new_solution[:nodes[0] + 1] + \
            [new_solution[nodes[1]]] + \
            new_solution[nodes[0] + 1 : nodes[1]] + \
            new_solution[nodes[1] +1 : ]
        
    # re-normalize
    idx_start = new_solution.index(0)
    new_solution = new_solution[idx_start:] + new_solution[:idx_start]
        
    return Individual(genotype=new_solution, weight=compute_weight(problem, new_solution))

def inversion_mutation(solution: Individual, problem: np.ndarray, n_mutations: int = 1) -> Individual:
    new_solution = solution.genotype.copy()
    for _ in range(n_mutations):
        nodes = random.sample(range(len(solution.genotype)), 2)
        nodes.sort()
        if nodes[0] != 0:
            new_solution = new_solution[:nodes[0]] + \
                new_solution[nodes[1]:nodes[0]-1:-1] + \
                new_solution[nodes[1]+1:]
        else:
            # if nodes[0] is 0 slicing with nodes[0]-1 is an error, so we handle this case separately
            new_solution = new_solution[nodes[1]::-1] + \
                new_solution[nodes[1]+1:]
    # re-normalize
    idx_start = new_solution.index(0)
    new_solution = new_solution[idx_start:] + new_solution[:idx_start]
        
    return Individual(genotype=new_solution, weight=compute_weight(problem, new_solution))

In [115]:
pop = create_population(test_problem)
ic(pop[0])
ic(inversion_mutation(pop[0], test_problem))

ic| pop[0]: individual(genotype=[0, 15, 14, 3, 5, 18, 17, 8, 10, 13, 11, 16, 1, 19, 6, 7, 12, 9, 4, 2], weight=np.float64(5080.56))
ic| new_solution: [5, 3, 14, 15, 0, 18, 17, 8, 10, 13, 11, 16, 1, 19, 6, 7, 12, 9, 4, 2]
ic| inversion_mutation(pop[0], test_problem): individual(genotype=[0, 18, 17, 8, 10, 13, 11, 16, 1, 19, 6, 7, 12, 9, 4, 2, 5, 3, 14, 15], weight=np.float64(5728.86))


individual(genotype=[0, 18, 17, 8, 10, 13, 11, 16, 1, 19, 6, 7, 12, 9, 4, 2, 5, 3, 14, 15], weight=np.float64(5728.86))

In [13]:
def tournament_selection(population: list, tournament_size: int = 2) -> Individual:
    tournament = random.sample(population, tournament_size)
    return min(tournament, key=lambda ind: ind.weight)

In [152]:
def print_test_solution(solution: list):
    for i, city_ind in enumerate(solution):
        print(f"{CITIES[city_ind]} --{test_problem[city_ind, solution[(i+1) % len(solution)]]}--> ", end="")
        if (i+1)%4==0:
            print()
    print(f"Total distance: {compute_weight(test_problem, solution)}")

In [157]:
# Create a population of random greedy solutions
def create_population(problem: np.ndarray, randomization: float = 0.2, population_size: int = POPULATION_SIZE) -> list:
    population = []
    for _ in range(population_size):
        start_node = random.randint(0, problem.shape[0]-1)
        solution = greedy_solution_2(problem, start_node, randomization)

        # After creating agreedy solution starting from the node start_node I want to "normalize" it
        # so that all solutions start from node 0, even though they are composed by different sequences of nodes
        idx_start = solution.index(0)
        solution = solution[idx_start:] + solution[:idx_start]
        
        # After that we can compute the weight
        weight = compute_weight(problem, solution)
        population.append(Individual(genotype=solution, weight=weight))

    return population


In [ ]:
def evolutionary_algorithm(problem: np.ndarray, 
                           generations: int = 1000,
                           mutation_rate: float = MUTATION_RATE,
                           initial_randomization: float = 0.2,
                           population_size: int = POPULATION_SIZE,
                           offspring_size: int = OFFSPRING_SIZE) -> list:
    
    population = create_population(problem, initial_randomization, population_size)
    best_solution = Individual(genotype=None, weight=float('inf'))

    # print(f"Initial best solution weight: {min(population, key=lambda ind: ind.weight).weight}")

    for generation in range(generations):
        offsprings = []

        # generatin of the offsprings
        for _ in range(offspring_size):
            if np.random.rand() < mutation_rate:
                # Mutation
                parent = tournament_selection(population, tournament_size=2)

                # offspring = mutation(parent, problem, n_mutations=random.randint(1,5))      # do from 1 up to 5 swaps in a mutation
                # offspring = insert_mutation(parent, problem, n_mutations=random.randint(1,5))      # do from 1 up to 5 swaps in a mutation

                offspring = inversion_mutation(parent, problem)     # the inversion mutation seems to work better than the others
            
            else:
                # Crossover
                parent1 = tournament_selection(population)
                parent2 = tournament_selection(population)
                offspring = crossover(parent1, parent2, problem)
                # ic(parent1, parent2, offspring)

            offsprings.append(offspring)

        # Add the offsprings to the population
        # Steady state approach
        population.extend(offsprings)
        
        # Sort the population by weight
        population.sort(key=lambda ind: ind.weight)
        population = population[:population_size]

        # Keep track of the best solution
        if population[0].weight < best_solution.weight:
            best_solution = population[0]

    return best_solution


In [ ]:
solution = evolutionary_algorithm(problem, generations=10000, mutation_rate=0.2, initial_randomization=0.2)
print(f"Best solution weight found: {solution.weight}")

In [199]:
for x in [10, 20, 50, 100, 200, 500, 1000]:
    problem = np.load('lab2/problem_g_' + str(x) + '.npy')
    solution = evolutionary_algorithm(problem, generations=100, mutation_rate=1, initial_randomization=0)
    print(f"G{x}:\tBest solution weight found: {solution.weight: .2f}")

G10:	Best solution weight found:  1497.66
G20:	Best solution weight found:  1755.51
G50:	Best solution weight found:  2916.53
G100:	Best solution weight found:  4371.87
G200:	Best solution weight found:  6244.22
G500:	Best solution weight found:  9655.16


KeyboardInterrupt: 

In [197]:
solution = evolutionary_algorithm(test_problem, generations=1000, mutation_rate=0.2, initial_randomization=0.2)
print(f"Best solution weight found: {solution.weight}")
print_test_solution(solution.genotype)

Best solution weight found: 2823.79
Rome --188.43--> Naples --314.03--> Palermo --165.64--> Catania --86.76--> 
Messina --291.45--> Taranto --79.15--> Bari --562.52--> Trieste --116.14--> 
Venice --34.44--> Padua --69.19--> Verona --61.86--> Brescia --80.1--> 
Milan --125.52--> Turin --123.92--> Genoa --117.9--> Parma --50.22--> 
Modena --37.14--> Bologna --71.26--> Prato --17.21--> Florence --230.91--> 
Total distance: 2823.79
